In [1]:
# Import the libraries
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.preprocessing.image import img_to_array, load_img


###Seeding

In [2]:
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

###Hyperparameters

Setting hyperparameters for the model

In [13]:
batch_size = 8
lr = 0.0001
epochs = 100
height = 256
width = 256

###Path

In [4]:
dataset_path = os.path.join('/content/drive/MyDrive/US dataset/Original without duplicated, misclassification and axila/Owdma')

##Unet

Defining the architecture of the U-Net model

In [5]:
def conv_block(inputs, num_filters):
  x = Conv2D(num_filters, 3, padding='same')(inputs)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  x = Conv2D(num_filters, 3, padding='same')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  return x

In [6]:
def encoder_block(inputs, num_filters):
  x = conv_block(inputs, num_filters)
  p = MaxPool2D((2, 2))(x)
  return x, p

In [7]:
def decoder_block(inputs, skip, num_filters):
  x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding='same')(inputs)
  x = Concatenate()([x, skip])
  x = conv_block(x, num_filters)

  return x

In [8]:
def build_unet(input_shape):
  inputs = Input(input_shape)

  """ Encoder"""
  s1, p1 = encoder_block(inputs, 64)
  s2, p2 = encoder_block(p1, 128)
  s3, p3 = encoder_block(p2, 256)
  s4, p4 = encoder_block(p3, 512)

  """ Bridge """
  b1 = conv_block(p4, 1024)

  """ Decoder """
  d1 = decoder_block(b1, s4, 512)
  d2 = decoder_block(d1, s3, 256)
  d3 = decoder_block(d2, s2, 128)
  d4 = decoder_block(d3, s1, 64)

  outputs = Conv2D(1, 1, padding='same', activation='sigmoid')(d4)

  model = Model(inputs, outputs, name='UNet')

  return model

###Pipeline

The following function is commented because after I have a function for images and another for masks. It is important because we need to do a train set and a test set.

In [14]:
#def load_and_preprocess_images(data_dir):
#   images = []
#    masks = []
#    labels = []

#    for class_name, class_label in class_labels.items():
#        class_dir = os.path.join(data_dir, class_name)
#        for image_file in os.listdir(class_dir):
#            if image_file.endswith(".png") and 'mask' not in image_file:
#                # Load the original image
#                image = img_to_array(load_img(os.path.join(class_dir, image_file), target_size=image_size))
#                image = image / 255.0  # Normalize pixel values

                # Find all masks associated with the image
#                image_name = os.path.splitext(image_file)[0]  # Remove the file extension
#                matching_masks = [f for f in os.listdir(class_dir) if f.startswith(image_name) and f.endswith("_mask.png") and '_1_mask' not in image_file and '_2_mask' not in image_file]

#                for mask_file in matching_masks:
#                    mask = img_to_array(load_img(os.path.join(class_dir, mask_file), target_size=image_size, color_mode="grayscale"))
#                    mask = mask / 255.0  # Normalize pixel values
#                    labels.append(class_label)

#                    images.append(image)
#                    masks.append(mask)

#    return images, masks, labels

In [15]:
# Load and preprocess the data using the defined function
data_dir = "/content/drive/MyDrive/US dataset/Original without duplicated, misclassification and axila/Owdma"
class_labels = {"normal": 0, "benign": 1, "malignant": 2}
image_size = (height, width)
#images, masks, labels = load_and_preprocess_images(data_dir)

In [16]:
def read_image(path):
  for class_name, class_label in class_labels.items():
        class_dir = os.path.join(data_dir, class_name)
        for image_file in os.listdir(class_dir):
            if image_file.endswith(".png") and 'mask' not in image_file:
                # Load the original image
                image = img_to_array(load_img(os.path.join(class_dir, image_file), target_size=image_size))
                image = image / 255.0  # Normalize pixel values
  return image

In [17]:
image = read_image(data_dir)

In [18]:
def read_mask(path):
  for class_name, class_label in class_labels.items():
        class_dir = os.path.join(data_dir, class_name)
        for image_file in os.listdir(class_dir):
          # Find all masks associated with the image
              image_name = os.path.splitext(image_file)[0]  # Remove the file extension
              matching_masks = [f for f in os.listdir(class_dir) if f.startswith(image_name) and f.endswith("_mask.png") and '_1_mask' not in image_file and '_2_mask' not in image_file]

              for mask_file in matching_masks:
                  mask = img_to_array(load_img(os.path.join(class_dir, mask_file), target_size=image_size, color_mode="grayscale"))
                  mask = mask / 255.0  # Normalize pixel values
  return mask

In [19]:
mask = read_mask(data_dir)

In [20]:
def train_valid_split(images, masks):
    num_samples = len(images)
    split_index = int(0.8 * num_samples)

    train_x = images[:split_index]
    valid_x = images[split_index:]

    train_y = masks[:split_index]
    valid_y = masks[split_index:]

    return train_x, valid_x, train_y, valid_y

In [21]:
def tf_parse(image, mask):
    def _parse(x, y):
      x = read_image(x)
      y = read_mask(y)
      return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([height, width, 3])
    y.set_shape([height, width, 1])

    return x, y

In [22]:
def tf_dataset(images, masks, batch=8):
  dataset = tf.data.Dataset.from_tensor_slices((images, masks))
  dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
  dataset = dataset.batch(batch)
  dataset = dataset.prefetch(tf.data.AUTOTUNE)
  return dataset

##Training

In [23]:
train_x, valid_x, train_y, valid_y = train_valid_split(images, masks)

In [24]:
print(len(train_x))
print(len(train_y))
print(len(valid_x))
print(len(valid_y))

439
439
110
110


In [ ]:

# Combine train_x and train_y into train_dataset
#train_dataset = pd.DataFrame({'images': train_x,
#                              'masks': train_y})

# Combine valid_x and valid_y into valid_dataset
#valid_dataset = pd.DataFrame({'images': valid_x,
#                              'masks': valid_y})

This is the code that I used to convert the np array to tensor. But it outputs a flattened array that cannot be used in the model. The code that is below is the one that the guy of the video used to transform the train and test set to tensors.

In [50]:
import pandas as pd
# Combine train_x and train_y into train_dataset
train_dataset = pd.DataFrame({'images': [x.flatten() for x in train_x],
                              'masks': [y.flatten() for y in train_y]})

# Combine valid_x and valid_y into valid_dataset
valid_dataset = pd.DataFrame({'images': [x.flatten() for x in valid_x],
                              'masks': [y.flatten() for y in valid_y]})

In [ ]:
train_images = tf.convert_to_tensor(train_dataset['images'].values.tolist())
train_masks = tf.convert_to_tensor(train_dataset['masks'].values.tolist())
valid_images = tf.convert_to_tensor(valid_dataset['images'].values.tolist())
valid_masks = tf.convert_to_tensor(valid_dataset['masks'].values.tolist())

# Create TensorFlow datasets
train_dataset_tf = tf.data.Dataset.from_tensor_slices((train_images, train_masks))
valid_dataset_tf = tf.data.Dataset.from_tensor_slices((valid_images, valid_masks))

In [43]:
input_shape = (height, width, 3)
model = build_unet(input_shape)

In [44]:
model.summary()

Model: "UNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d_19 (Conv2D)          (None, 512, 512, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_18 (Ba  (None, 512, 512, 64)         256       ['conv2d_19[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_18 (Activation)  (None, 512, 512, 64)         0         ['batch_normalization_18[0]

In [45]:
opt = tf.keras.optimizers.Adam(lr)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['acc'])

The code that follows uses: (1) reduce LR on plateau: when the loss function on the validation set does not imporve in 4 epochs, the LR is reduced by a factor of 0.1. And (2) the early stopping if in 20 epochs the loss function on the validation set does not improve, it stops the algorithm.

In [46]:
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
]

In [47]:
model.fit(
    train_dataset_tf,
    validation_data=valid_dataset_tf,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/100


ValueError: ignored